In [1]:
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

import scipy.stats as stats

In [2]:
def _internal_runner(times_1, times_2, delta_t):
    coincidences = 0
    
    for time_1 in times_1:
        diff_matrix_count = np.nansum(np.abs(times_2 - time_1) <= delta_t)
        
        if diff_matrix_count > 0:
            coincidences += 1
    
    return coincidences

def simulate_events(lambda_G, lambda_S, delta_t):
    """
    Simulate the occurrence of events from signal G and signal S
    """
    
    n_G = np.random.poisson(lambda_G)
    n_S = np.random.poisson(lambda_S)
    
    times_G = np.random.uniform(0, 1, n_G)
    times_S = np.random.uniform(0, 1, n_S)
    
    g_coincidence = _internal_runner(times_G, times_S, delta_t)
    g_denominator = len(times_G)
    
    s_coincidence = _internal_runner(times_S, times_G, delta_t)
    s_denominator = len(times_S)
    
    a_denominator = len(times_S) + len(times_G)

    g_val = g_coincidence/g_denominator
    s_val = s_coincidence/s_denominator
    a_val = (g_coincidence + s_coincidence)/a_denominator
    
    return g_val, s_val, a_val

def perform_simulations(lambda_G, lambda_S, delta_t, num_simulations):
    """
    Run the simulation multiple times and calculate the probability of coincidences
    """

    g_vals = 0
    s_vals = 0
    a_vals = 0
    
    for _ in tqdm(range(num_simulations), desc='Simulations'):
        g_val, s_val, a_val = simulate_events(lambda_G, lambda_S, delta_t)
        g_vals += g_val
        s_vals += s_val
        a_vals += a_val
    
    return g_vals/num_simulations, s_vals/num_simulations, a_vals/num_simulations


In [3]:
lambda_G = 2132.0/14  # rate of signal G (events per year)
# lambda_S = 487.62516272856794
lambda_S = 6368/3 * 0.21
# delta_t = 0.0833333333333/365.25/2   # uncertainty window in years
# delta_t = 0.3/365.25   # uncertainty window in years
delta_t = 0.15/365.25   # uncertainty window in years
num_simulations = 10000  # number of simulations

In [4]:
grb_prob, sne_prob, all_prob = perform_simulations(lambda_G, lambda_S, delta_t, num_simulations)

print(f"Probability of a GRB Coinc. occurring: {grb_prob*100:.2f}") # Display the probability
print(f"Probability of a SNe Coinc. occurring: {sne_prob*100:.2f}") # Display the probability
print(f"Probability of a all Coinc. occurring: {all_prob*100:.2f}") # Display the probability

Simulations: 100%|██████████| 10000/10000 [00:31<00:00, 315.50it/s]

Probability of a GRB Coinc. occurring: 30.62
Probability of a SNe Coinc. occurring: 11.77
Probability of a all Coinc. occurring: 16.55


In [6]:
symm = ( lambda_G *( 1 - np.exp(- 2*delta_t * lambda_S) ) +  lambda_S*( 1 - np.exp(- 2*delta_t * lambda_G) ) ) / (lambda_S + lambda_G)

print(f"Probability of a Symm. Coinc. occurring: {symm*100:.2f}") # Display the probability

Probability of a Symm. Coinc. occurring: 16.57


In [11]:
for prob in [grb_prob, sne_prob, all_prob, symm]:
    cL = 1 - prob
    z_score = stats.norm.ppf((1 + cL) / 2)

    print(f"The z-score for a {cL*100:.3f}% confidence level is: {z_score:.3f}")

The z-score for a 69.382% confidence level is: 1.023
The z-score for a 88.231% confidence level is: 1.565
The z-score for a 83.451% confidence level is: 1.387
The z-score for a 83.430% confidence level is: 1.386


In [10]:
1 - np.exp(-2*delta_t*lambda_S)

0.3065854830547303

In [15]:


cL = 1 - 0.15321393130481006
z_score = stats.norm.ppf((1 + cL) / 2)

print(f"The z-score for a {cL*100:.3f}% confidence level is: {z_score:.3f}")

The z-score for a 84.679% confidence level is: 1.428


In [13]:
0.07859334672918063*100

7.859334672918063

In [14]:
all_prob

0.16548544904580562